## **Imports**

In [2]:
from google.colab import files

import glob
import os
import librosa

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from sklearn.model_selection import train_test_split
import numpy as np

## **Data Download Directory Setup**

In [ ]:
# Download is large - this may take a while to run (assuming the data is not already available)

files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d pythonafroz/electrical-motor-anomaly-detection-from-sound-data

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/pythonafroz/electrical-motor-anomaly-detection-from-sound-data
License(s): CC-BY-NC-SA-4.0
 99% 4.13G/4.16G [01:29<00:00, 113MB/s] 
100% 4.16G/4.16G [01:30<00:00, 49.6MB/s]


In [ ]:
# Extract Zip file contents

!unzip electrical-motor-anomaly-detection-from-sound-data.zip -d motor_data

## **MFCC Data Conversion**

In [ ]:
# Start with aptly provided training data from set

wav_files_train = glob.glob("/content/motor_data/dev_data_fan/fan/train/*.wav", recursive=True)
print("Total training files:", len(wav_files_train))

wav_files_test_source = glob.glob("/content/motor_data/dev_data_fan/fan/source_test/*.wav", recursive=True)
print("Total testing files (source domain):", len(wav_files_test_source))

wav_files_test_target = glob.glob("/content/motor_data/dev_data_fan/fan/target_test/*.wav", recursive=True)
print("Total testing files (target domain):", len(wav_files_test_target))

"""
Converts a WAV file to an MFCC representation for easy usage within sequential
model training.
"""
def wav_to_mfcc(path, sr=16000, duration=2.0, n_mfcc=40):
    # Load audio file
    y, _ = librosa.load(path, sr=sr)

    # Fix length to 2 seconds
    samples = int(sr * duration)
    y = librosa.util.fix_length(data=y, size=samples)

    # Compute MFCC
    mfcc = librosa.feature.mfcc(
        y=y,
        sr=sr,
        n_mfcc=n_mfcc
    )

    # Add channel dimension (n_mfcc, time, 1)
    mfcc = mfcc[..., np.newaxis]

    return mfcc

# X training data
x_train = []
for f in wav_files_train:
    mfcc = wav_to_mfcc(f)
    x_train.append(mfcc)
x_train = np.array(x_train)
print("X shape:", x_train.shape)

# Y1 target domain test data
y1_test = []
for f in wav_files_test_target:
    mfcc = wav_to_mfcc(f)
    y1_test.append(mfcc)
y1_test = np.array(y1_test)
print("Y1 shape:", y1_test.shape)

# Y2 source domain test data
y2_test = []
for f in wav_files_test_source:
    mfcc = wav_to_mfcc(f)
    y2_test.append(mfcc)
y2_test = np.array(y2_test)
print("Y2 shape:", y2_test.shape)

Total training files: 3009
Total testing files (source domain): 600
Total testing files (target domain): 600
X shape: (3009, 40, 63, 1)
Y1 shape: (600, 40, 63, 1)
Y2 shape: (600, 40, 63, 1)


## **Model Setup & Training**

In [ ]:
def pad_mfcc(X):
    H, W = X.shape[1:3]
    pad_h = (4 - H % 4) % 4
    pad_w = (4 - W % 4) % 4
    return np.pad(X, ((0,0), (0,pad_h), (0,pad_w), (0,0)), mode='constant')

x_train = pad_mfcc(x_train)

X_train, X_val = train_test_split(
    x_train, test_size=0.2, shuffle=True, random_state=42
)

print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)

input_shape = X_train.shape[1:]

# small conv autoencoder
inp = layers.Input(shape=input_shape)
x = layers.Conv2D(16, (3,3), activation='relu', padding='same', strides=2)(inp)
x = layers.Conv2D(32, (3,3), activation='relu', padding='same', strides=2)(x)
x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(x)

# bottleneck
x = layers.GlobalAveragePooling2D()(x)
bottleneck = layers.Dense(32, activation='relu')(x)

# decoder
x = layers.Dense((input_shape[0]//4)*(input_shape[1]//4)*64, activation='relu')(bottleneck)
x = layers.Reshape((input_shape[0]//4, input_shape[1]//4, 64))(x)
x = layers.Conv2DTranspose(32, (3,3), strides=2, padding='same', activation='relu')(x)
x = layers.Conv2DTranspose(16, (3,3), strides=2, padding='same', activation='relu')(x)
out = layers.Conv2D(1, (3,3), activation='linear', padding='same')(x)

autoencoder = models.Model(inputs=inp, outputs=out)
autoencoder.compile(optimizer='adam', loss='mse')

# train (only normal data)
autoencoder.fit(X_train, X_train,
                validation_data=(X_val, X_val),
                epochs=40, batch_size=16)

X_train shape: (2407, 40, 64, 1)
X_val shape: (602, 40, 64, 1)
Epoch 1/40
151/151 ━━━━━━━━━━━━━━━━━━━━ 14s 64ms/step - loss: 760.1630 - val_loss: 38.9222
Epoch 2/40
151/151 ━━━━━━━━━━━━━━━━━━━━ 9s 61ms/step - loss: 36.2701 - val_loss: 34.4786
Epoch 3/40
151/151 ━━━━━━━━━━━━━━━━━━━━ 10s 65ms/step - loss: 33.4247 - val_loss: 33.1535
Epoch 4/40
151/151 ━━━━━━━━━━━━━━━━━━━━ 10s 66ms/step - loss: 32.1531 - val_loss: 33.3245
Epoch 5/40
151/151 ━━━━━━━━━━━━━━━━━━━━ 9s 61ms/step - loss: 32.7640 - val_loss: 32.4711
Epoch 6/40
151/151 ━━━━━━━━━━━━━━━━━━━━ 9s 62ms/step - loss: 31.9514 - val_loss: 32.2750
Epoch 7/40
151/151 ━━━━━━━━━━━━━━━━━━━━ 10s 66ms/step - loss: 31.3648 - val_loss: 32.4451
Epoch 8/40
151/151 ━━━━━━━━━━━━━━━━━━━━ 10s 65ms/step - loss: 32.1007 - val_loss: 32.3459
Epoch 9/40
151/151 ━━━━━━━━━━━━━━━━━━━━ 9s 62ms/step - loss: 31.8728 - val_loss: 32.1590
Epoch 10/40
151/151 ━━━━━━━━━━━━━━━━━━━━ 9s 61ms/step - loss: 31.1417 - val_loss: 32.0133
Epoch 11/40
151/151 ━━━━━━━━━━━━━━━━━━━━

In [1]:
# Get params, model summary

autoencoder.summary()

NameError: name 'autoencoder' is not defined

In [ ]:
# Calculate the threshold source - very important for inference
# Always varies from device to device, this is just a quick test
# I would not use this threshold automatically for deployment, but it is a good starting point
recons = autoencoder.predict(X_val)
val_errors = np.mean((recons - X_val)**2, axis=(1,2,3))

# Use 95th percentile; above = anomalous, below = normal
threshold_source = np.percentile(val_errors, 95)

print(threshold_source)

19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step
32.873253


## **Quantization / Compression**

In [ ]:
model_path = "autoencoder_baseline.keras"
autoencoder.save(model_path)

# Convert baseline to INT8 quantized
converter = tf.lite.TFLiteConverter.from_keras_model(autoencoder)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

quant_model_path = "autoencoder_int8.tflite"
with open(quant_model_path, "wb") as f:
    f.write(tflite_model)

original_size = os.path.getsize(model_path) / 1e6
quant_size = os.path.getsize(quant_model_path) / 1e6

print(f"Original model size: {original_size:.2f} MB")
print(f"Quantized INT8 model size: {quant_size:.2f} MB")
print(f"Size reduction: {100*(original_size - quant_size)/original_size:.1f}%")

Saved artifact at '/tmp/tmpacnslq2p'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 40, 64, 1), dtype=tf.float32, name='keras_tensor_55')
Output Type:
  TensorSpec(shape=(None, 40, 64, 1), dtype=tf.float32, name=None)
Captures:
  132402193920656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132402193921808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132402193922960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132402193922192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132402193923344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132402193923920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132402193921616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132402193921424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132402193922768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132402193922576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13240